In [1]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict
import gensim
import datetime
from datetime import datetime as dt, timedelta as td
import re
import textstat
from textstat.textstat import textstat
import nltk
from itertools import chain
import pandasql as ps
from BaseAD import CreateBaseAD,QuarterIndex ,get_quarteryr
from qaextraction import get_analysts_from_preamble
import pandas as pd
import itertools
import time
from utils import get_date_from_preamble

# load_ext autoreload

# autoreload 2

/home/affine/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


None
None


In [2]:
data = pd.read_csv('/media/affine/New Volume/Chubb/Data/seekingAlpha_transcripts_483_companies_combined.txt', sep="|")
# print(data.columns)
# print(data.head)
# print(data.count)

In [3]:
data = data.drop_duplicates()
#len(data)

In [6]:

data = data[~data['Preamble'].isnull()]
len(data)
data

input company name                                    About  \
0                   AMZN                                      NaN   
1                   AMZN                  Amazon.com, Inc. (AMZN)   
2                   AMZN                  Amazon.com, Inc. (AMZN)   
3                   AMZN                  Amazon.com, Inc. (AMZN)   
4                   AMZN                  Amazon.com, Inc. (AMZN)   
5                   AMZN                  Amazon.com, Inc. (AMZN)   
6                   AMZN                  Amazon.com, Inc. (AMZN)   
7                   AMZN                  Amazon.com, Inc. (AMZN)   
8                   AMZN                  Amazon.com, Inc. (AMZN)   
9                   AMZN                  Amazon.com, Inc. (AMZN)   
10                  AMZN                  Amazon.com, Inc. (AMZN)   
11                  AMZN                  Amazon.com, Inc. (AMZN)   
12                  AMZN                  Amazon.com, Inc. (AMZN)   
13                  AMZN                  Amazon.com, Inc. (AMZN)   
14                  AMZN                  Amazon.com, Inc. (AMZN)   
15                  AMZN                  Amazon.com, Inc. (AMZN)   
16                  AMZN                  Amazon.com, Inc. (AMZN)   
17                  AMZN                  Amazon.com, Inc. (AMZN)   
18                  AMZN                  Amazon.com, Inc. (AMZN)   
19                  AMZN                  Amazon.com, Inc. (AMZN)   
20                  AMZN                  Amazon.com, Inc. (AMZN)   
21                  AMZN                  Amazon.com, Inc. (AMZN)   
22                  AMZN                  Amazon.com, Inc. (AMZN)   
23                  AMZN                  Amazon.com, Inc. (AMZN)   
24                  AMZN                  Amazon.com, Inc. (AMZN)   
25                  AMZN                  Amazon.com, Inc. (AMZN)   
26                  AMZN                  Amazon.com, Inc. (AMZN)   
27                  AMZN                  Amazon.com, Inc. (AMZN)   
28                  AMZN                  Amazon.com, Inc. (AMZN)   
29                  AMZN                  Amazon.com, Inc. (AMZN)   
...                  ...                                      ...   
16492                MSG  The Madison Square Garden Company (MSG)   
16493                MSG  The Madison Square Garden Company (MSG)   
16494                MSG  The Madison Square Garden Company (MSG)   
16495                MSG  The Madison Square Garden Company (MSG)   
16496                MSG  The Madison Square Garden Company (MSG)   
16497                MSG  The Madison Square Garden Company (MSG)   
16498               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16499               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16500               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16501               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16502               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16503               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16504               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16505               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16506               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16507               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16508               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16509               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16510               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16511               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16512               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16513               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16514               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16515               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16516               UBNT           Ubiquiti Networks, Inc. (UBNT)   
16517               UBNT           Ubiquiti Networks, Inc. (UBNT)   
1651

In [5]:
# Extract date from preamble
data['date_earning_call'] = data.apply( lambda x: get_date_from_preamble(x['Preamble'], x['Date published'], print_error=True), axis=1)

couldn't find date <preamble> Altera Corp. (NASDAQ:ALTR)^Q2 2006 Earnings Conference Call^July 24, 2006  4.45 pm EST^ ~Executives~ </preamble>,                  returning fallback_date: 2006-07-25 00:31:44
couldn't find date <preamble> Start Time: 18:00 January  1, 0000  6:41 PM ET^Netflix, Inc. (NASDAQ:NFLX)^Q4 2017 Earnings Conferen </preamble>,                  returning fallback_date: 2018-01-23 04:43:05
couldn't find date <preamble>  ~Executives~ ^Cynthia He - Investor Relations Manager ^ Robin Li - Chief Executive Officer ^ Shawn  </preamble>,                  returning fallback_date: 2006-02-22 11:09:11
couldn't find date <preamble>  ~Executives~ ^Alexandra Deignan - IR^Tamara Lundgren - President and CEO^Richard Peach - SVP and CF </preamble>,                  returning fallback_date: 2012-01-09 20:40:19
couldn't find date <preamble> Start Time: 16:30 January  1, 0000  5:07 PM ET^Alnylam Pharmaceuticals, Inc. (NASDAQ:ALNY)^Q1 2013 E </preamble>,                  returning fallb

In [7]:
# Convert into date level
data["published_date_time"] = data["Date published"].apply(pd.to_datetime)

In [8]:
#EDA KIND
# Create key at date and company level
data['date_earning_call_str'] = [str(i) for i in list(data['date_earning_call'])]
data['DateEarningQuarterYr'] = data['date_earning_call_str'].apply(get_quarteryr)
data['key'] = data['input company name'] + data['date_earning_call_str']

In [9]:

data['Count'] = data.groupby('key')['key'].transform('count')
data['max_published_date'] = data.groupby(['key'])['published_date_time'].transform('max')
data_temp = data[data['Count']>1 ]
data_temp2 = data_temp[data_temp['published_date_time'] == data_temp['max_published_date']]
data_temp1 = data[data['Count']==1]
# Remove duplicates and final data
data_final = data_temp1.append(data_temp2,ignore_index =True)


In [19]:
#Key at Quarter and company level
data_final['key_quarter'] = data_final['input company name'] + data_final['DateEarningQuarterYr']
data_final['Count_quarter'] = data_final.groupby('key_quarter')['key_quarter'].transform('count')
data_final['max_published_date'] = data_final.groupby(['key_quarter'])['published_date_time'].transform('max')


In [20]:
print(set(data_final['Count_quarter']))
print(len(data_final[data_final['Count_quarter']==2]))
print(len(data_final[data_final['Count_quarter']==3]))

{1, 2, 3}
1340
60


In [22]:
pd.DataFrame(data_final[['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Date published',
       
       'date_earning_call', 'published_date_time', 'date_earning_call_str',
       'DateEarningQuarterYr', 'key', 'Count', 'max_published_date',
       'merge_flag', 'key_quarter', 'Count_quarter']]).to_csv('check_duplicates.csv')

In [88]:
fuzz.partial_ratio("Microsoft Corporation", "Microsoft Corporation F4Q10 (Qtr End 06/30/10) Earnings Call Transcript") > 90

True

In [117]:
def filter_merger_acquisition(row):
    company_name = re.sub(r"\W*" + str(row['input company name'])+ r"\W*", '',  str(row['About']))
    return (str(row["input company name"]) in str(row["Transcript page heading"])) | (fuzz.partial_ratio(company_name, str(row["Transcript page heading"])) > 90)

#data.columns
data_final["merge_flag"] = data_final.apply(filter_merger_acquisition, axis=1)

In [131]:
data_final

input company name                                About  \
0                   AMZN                                  NaN   
1                   AMZN              Amazon.com, Inc. (AMZN)   
2                   AMZN              Amazon.com, Inc. (AMZN)   
3                   AMZN              Amazon.com, Inc. (AMZN)   
4                   AMZN              Amazon.com, Inc. (AMZN)   
5                   AMZN              Amazon.com, Inc. (AMZN)   
6                   AMZN              Amazon.com, Inc. (AMZN)   
7                   AMZN              Amazon.com, Inc. (AMZN)   
8                   AMZN              Amazon.com, Inc. (AMZN)   
9                   AMZN              Amazon.com, Inc. (AMZN)   
10                  AMZN              Amazon.com, Inc. (AMZN)   
11                  AMZN              Amazon.com, Inc. (AMZN)   
12                  AMZN              Amazon.com, Inc. (AMZN)   
13                  AMZN              Amazon.com, Inc. (AMZN)   
14                  AMZN              Amazon.com, Inc. (AMZN)   
15                  AMZN              Amazon.com, Inc. (AMZN)   
16                  AMZN              Amazon.com, Inc. (AMZN)   
17                  AMZN              Amazon.com, Inc. (AMZN)   
18                  AMZN              Amazon.com, Inc. (AMZN)   
19                  AMZN              Amazon.com, Inc. (AMZN)   
20                  AMZN              Amazon.com, Inc. (AMZN)   
21                  AMZN              Amazon.com, Inc. (AMZN)   
22                  AMZN              Amazon.com, Inc. (AMZN)   
23                  AMZN              Amazon.com, Inc. (AMZN)   
24                  AMZN              Amazon.com, Inc. (AMZN)   
25                  AMZN              Amazon.com, Inc. (AMZN)   
26                  AMZN              Amazon.com, Inc. (AMZN)   
27                  AMZN              Amazon.com, Inc. (AMZN)   
28                  AMZN              Amazon.com, Inc. (AMZN)   
29                  AMZN              Amazon.com, Inc. (AMZN)   
...                  ...                                  ...   
16441               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16442               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16443               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16444               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16445               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16446               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16447               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16448               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16449               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16450               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16451               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16452               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16453               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16454               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16455               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16456               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16457               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16458               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16459               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16460               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16461               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16462               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16463               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16464               UBNT       Ubiquiti Networks, Inc. (UBNT)   
16465               GOOG           Alphabet, Inc. Cl C (GOOG)   
16466               GOOG           Alphabet, Inc. Cl C (GOOG)   
16467               FLIR            FLIR Systems, Inc. (FLIR)   
16468               CDNS  Cadence Design Systems, Inc. (CDNS)   
16469               RCII            Rent-A-Center Inc. (RCII)   
16470               RCII                 

In [111]:
data_final[data_final["merge_flag"] == True].groupby('key_quarter').agg({'key_quarter': 'count'})
flagged = data_final[data_final["merge_flag"] == True]
flagged['count'] = flagged.groupby('key_quarter')['key_quarter'].transform('count')

/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [112]:
flagged.to_csv('check.csv')

In [69]:
re.sub(r"\W*" + str('INT')+ r"\W*", '',  "dsadsa INT")
# data_final

'dsadsa'

In [80]:
data_final_temp1 = data_final[data_final['Count_quarter']==1]
data_final_temp2 = data_final[data_final['Count_quarter']>1]
data_final_temp3 = data_final_temp2[data_final['merge_flag']==True]
data_final2 =data_final_temp1.append(data_final_temp3,ignore_index =True)


/home/affine/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [81]:
print(len(data_final_temp1))
print(len(data_final_temp2))
print(len(data_final_temp3))
print(len(data_final2))

15071
1400
591
15662


In [101]:
# data_final['Count_quarter2'] = data_final.groupby('key_quarter')['key_quarter'].transform('count')
data_final.groupby('key_quarter')['key_quarter'].agg('count').

In [102]:
data_final[data_final['Count_quarter2'] > 2]

input company name                               About  \
71                  AMZN             Amazon.com, Inc. (AMZN)   
73                  AMZN             Amazon.com, Inc. (AMZN)   
74                  AMZN             Amazon.com, Inc. (AMZN)   
79                  AMZN             Amazon.com, Inc. (AMZN)   
81                  AMZN             Amazon.com, Inc. (AMZN)   
82                  AMZN             Amazon.com, Inc. (AMZN)   
146                  WDC   Western Digital Corporation (WDC)   
147                  WDC   Western Digital Corporation (WDC)   
154                  WDC   Western Digital Corporation (WDC)   
156                  WDC   Western Digital Corporation (WDC)   
158                  WDC   Western Digital Corporation (WDC)   
159                  WDC   Western Digital Corporation (WDC)   
166                  WDC   Western Digital Corporation (WDC)   
168                  WDC   Western Digital Corporation (WDC)   
169                  WDC   Western Digital Corporation (WDC)   
283                 INTC            Intel Corporation (INTC)   
290                 INTC            Intel Corporation (INTC)   
291                 INTC            Intel Corporation (INTC)   
373                 GOOG          Alphabet, Inc. Cl C (GOOG)   
374                 GOOG          Alphabet, Inc. Cl C (GOOG)   
375                 GOOG          Alphabet, Inc. Cl C (GOOG)   
382                 GOOG          Alphabet, Inc. Cl C (GOOG)   
429                 MSFT        Microsoft Corporation (MSFT)   
437                 MSFT        Microsoft Corporation (MSFT)   
438                 MSFT        Microsoft Corporation (MSFT)   
709                 HOLX                Hologic, Inc. (HOLX)   
710                 HOLX                Hologic, Inc. (HOLX)   
711                 HOLX                Hologic, Inc. (HOLX)   
836                 HOPE           Hope Bancorp, Inc. (HOPE)   
842                 HOPE           Hope Bancorp, Inc. (HOPE)   
843                 HOPE           Hope Bancorp, Inc. (HOPE)   
1359                NUAN  Nuance Communications, Inc. (NUAN)   
1361                NUAN  Nuance Communications, Inc. (NUAN)   
1362                NUAN  Nuance Communications, Inc. (NUAN)   
1370                NUAN  Nuance Communications, Inc. (NUAN)   
1371                NUAN  Nuance Communications, Inc. (NUAN)   
1372                NUAN  Nuance Communications, Inc. (NUAN)   
1746                BIDU                  Baidu, Inc. (BIDU)   
1747                BIDU                  Baidu, Inc. (BIDU)   
1750                BIDU                  Baidu, Inc. (BIDU)   
3362                ANGO          AngioDynamics, Inc. (ANGO)   
3365                ANGO          AngioDynamics, Inc. (ANGO)   
3366                ANGO          AngioDynamics, Inc. (ANGO)   
5676                LNDC           Landec Corporation (LNDC)   
5679                LNDC           Landec Corporation (LNDC)   
5680                LNDC           Landec Corporation (LNDC)   
6436                CREE                   Cree, Inc. (CREE)   
6437                CREE                   Cree, Inc. (CREE)   
6440                CREE                   Cree, Inc. (CREE)   
7865                CRUS           Cirrus Logic, Inc. (CRUS)   
7867                CRUS           Cirrus Logic, Inc. (CRUS)   
7868                CRUS           Cirrus Logic, Inc. (CRUS)   
9755                ARRY         Array BioPharma Inc. (ARRY)   
9756                ARRY         Array BioPharma Inc. (ARRY)   
9757                ARRY         Array BioPharma Inc. (ARRY)   
14388               ROLL    RBC Bearings Incorporated (ROLL)   
14389               ROLL    RBC Bearings Incorporated (ROLL)   
14390               ROLL    RBC Bearings Incorporated (ROLL)   
16465               GOOG          Alphabet, Inc. Cl C (GOOG)   
16466               GOOG          Alphabet, Inc. Cl C (GOOG)   

                                 Transcript page heading  \
71     Whole Foods Market, Inc. F4Q08 (Qt

In [99]:
data_final.groupby('key_quarter')['key_quarter'].agg('count').value_counts()

1    15071
2      670
3       20
Name: key_quarter, dtype: int64

In [83]:
data_final2.groupby('key_quarter')['key_quarter'].agg('count')

array([1, 2, 3])

In [73]:
data_final2[['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Date published',
       
       'date_earning_call', 'published_date_time', 'date_earning_call_str',
       'DateEarningQuarterYr', 'key', 'Count', 'max_published_date',
       'merge_flag', 'key_quarter', 'Count_quarter','Count_quarter2']].to_csv('check3_duplicates.csv')

In [79]:
len(data_final2[data_final2['Count_quarter2']==2])



0

In [75]:
# # Temporarily
# remove_companies = ['AMZN','WDC','MAR','INTC','GOOG','MSFT','CSCO','CY','CHTR','HOLX','SIRI','WLTW','HOPE','CMCSA',
#                     'COST','NUAN','SYMC','AMD','FISV','NVDA','PEP','CAR','CTAS','MCHP','MU','PAYX','CME','GILD','XEL',
#                     'ANGO','BBBY','RMBS','SONC','TSLA','AMTD','NATI','PCAR','SYNT','FLIR','KLAC','LNDC','MDRX','MNST',
#                     'STLD','AEIS','CAMP','CENX','MRVL','SCHL','SPWR','AVT','CDNS','RICK','CRUS','SBGI','BRKR','GNTX',
#                     'SCHN','TTWO','ABMD','ALNY','NKTR','HSII','PRGS','RCII','ACAD','AVAV','GPOR','MLHR','UNFI','AFSI',
#                     'FTNT','CONN','SCVL','VNDA','EHTH','GIII','SBRA','TRS','EEFT','EXAS','GRPN','PERY','SHLM','ROLL',
#                     'RDWR','EGAN','VRNT','CROX']



In [76]:
# data = data.loc[~data['input company name'].isin(remove_companies)]

In [18]:
data["next_publish_date"] =  data.sort_values(by='published_date_time').groupby(['input company name'])['published_date_time'].shift(-1)

## Defining default value of date 

data["def_date"] = datetime.datetime.strptime('2099-01-01', '%Y-%M-%d')

## Impute next publish date with def date in case of nulls 


data["next_publish_date"] = data.next_publish_date.combine_first(data.def_date)

## Read stanford CALS data 
cals_data = pd.read_csv('/media/affine/New Volume/Chubb/20180807/transcript_with_cals.csv')


## Double check the filing dates

cals_data["filing_date_2"] = cals_data["Filing Date"].apply(pd.to_datetime)

## Create flag for CALS

cals_data["is_cals"] = 1
         

In [19]:
quarter_index = QuarterIndex()


In [20]:
## Pandas SQL to implement Left Join like behavior with inequality join

sql_query = '''Select a.*, b.filing_date_2, coalesce(b.is_cals,0) as is_cals FROM data a left join 
cals_data b ON a."input company name" = b.Ticker 
AND a.published_date_time < b.filing_date_2 AND a.next_publish_date >= b.filing_date_2'''

data2 = ps.sqldf(sql_query)


In [ ]:
# Difference betweben filing date and transcript published date
#data2["published_date_time"] = data2["published_date_time"].apply(pd.to_datetime)

#data2["date_earning_call"] = data2["date_earning_call"].apply(pd.to_datetime)
for i in range(0,len(data2["date_earning_call"])):
    pd.to_datetime(data2["date_earning_call"].values[i])
    print(i)

    


In [80]:
%load_ext autoreload
%autoreload 2
from utils import DateFinder, get_date_from_preamble

[autoreload of regex failed: Traceback (most recent call last):
  File "/home/affine/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
TypeError: optimise() missing 1 required positional argument: 'reverse'
]


0305-08-13 00:00:00
None
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
list(DateFinder.parse_dates(data['Preamble'].values[642], source=True, strict=True))

[(datetime.datetime(2006, 5, 9, 20, 0), 'May 9, 2006 8:00 p.m. EST'),
 (datetime.datetime(2006, 1, 1, 0, 0), 'on January 1, 2006')]

In [79]:
get_date_from_preamble("305")

datetime.datetime(305, 8, 13, 0, 0)

In [56]:
data[['Preamble', 'Date published']].values[i]

array([" ~Executives~ ^Cynthia He - Investor Relations Manager ^ Robin Li - Chief Executive Officer ^ Shawn Wang - Chief Financial Officer ^ ~Analysts~ ^Jason Brueschke - Citigroup ^ Anthony Noto - Goldman Sachs ^ Safa Rashtchy - Piper Jaffray ^ Richard Ji - Morgan Stanley ^ James Mitchell - Goldman Sachs ^ Leah Hao - Thomas Weisel Partners ^ Tan Ke - Owens Technology Research ^ Robert Peck - Bear Stearns ^ Tony Tang - Lucite Research ^ Robert King - Peninsula Capital ^ Wallace Cheung - CSFB ^ Andrew Collier - New York Global Securities ^ ~Operator~ ^Good day ladies and gentlemen and welcome to Baidu's fourth quarter and full year 2005 conference call. (Operator instructions) I would now like to turn the meeting over to your host for today's conference, Ms. Cynthia He, Baidu's Investor Relations Manager. Please proceed. ^ ~Cynthia He~ ^Thank you. Hello, everyone and welcome to Baidu's fourth quarter and fiscal year 2005 earnings conference call. My name is Cynthia He, Baidu's IR Manage

In [81]:

data["date_earning_call"].values[i]    

datetime.datetime(305, 2, 22, 11, 9, 11)

In [ ]:
data2["filing_date_2"] = data2["filing_date_2"].apply(lambda x: pd.to_datetime(x) if x!=None else datetime.datetime(2099, 1, 1))
#data2['data_diff_temp'] = (data2['filing_date_2'].values -  data2['published_date_time'].values)
data2['data_diff_temp'] = (data2['filing_date_2'].values -  data2['date_earning_call'].values)
data2['data_diff'] = data2['data_diff_temp'].dt.days
#data2['QuarterYr'] = data2['Date published'].apply(get_quarteryr)
data2['Next_QuarterYr'] = data2['next_publish_date'].apply(get_quarteryr)

data2["next_publish_date"] = data2["next_publish_date"].apply(lambda x: pd.to_datetime(x) if x!=None else datetime.datetime(2099, 1, 1))



data2['next_currt_diff_temp'] = (data2['next_publish_date'].values -  data2['date_earning_call'].values)

data2['next_currt_diff'] = data2['next_currt_diff_temp'].dt.days

#data2['Actual_QuarterYr'] = 'none'   
# for i in range(0,len(data2)):
# #    if data2['data_diff'].values[i] <120 and data2['data_diff'].values[i]>0 and data2['next_currt_diff'].values[i] <120 and data2['next_currt_diff'].values[i]>0 or data2['next_publish_date'].values[i] == '2099-01-01' :
#     if data2['data_diff'].values[i]>0 and data2['next_currt_diff'].values[i] <120 and data2['next_currt_diff'].values[i]>0 or str(data2['next_publish_date'].values[i]) == '2099-01-01T00:01:00.000000000'  :
#         data2['Actual_QuarterYr'].values[i] =    data2['QuarterYr'].values[i]
#     else:
#         data2['Actual_QuarterYr'].values[i] =    data2['Next_QuarterYr'].values[i]

for i in range(0,len(data2)):
    if  data2['is_cals'].values[i]==0:
        data2['Actual_QuarterYr'].values[i] =    data2['QuarterYr'].values[i]
    elif data2['data_diff'].values[i]>0 and data2['next_currt_diff'].values[i] <120 and data2['next_currt_diff'].values[i]>0 or str(data2['next_publish_date'].values[i]) == '2099-01-01T00:01:00.000000000'  :

        data2['Actual_QuarterYr'].values[i] =    data2['QuarterYr'].values[i]
        
    else:
        data2['Actual_QuarterYr'].values[i] =    data2['Next_QuarterYr'].values[i]
       


In [22]:
data.columns

Index(['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Date published', 'Preamble',
       'transcript after preamble', 'Transcript link', 'Scraped date',
       'date_earning_call', 'published_date_time', 'date_earning_call_str',
       'DateEarningQuarterYr', 'key', 'Count', 'max_published_date',
       'key_quarter', 'Count_quarter', 'next_publish_date', 'def_date'],
      dtype='object')

In [30]:
data4 = pd.merge(data2,quarter_index,how = 'left',left_on = 'DateEarningQuarterYr',right_on = 'QuarterYr')
data4['Previous_QuarterNbr'] = data4['QuarterNbr'] - 1
#data4 = data4.rename(columns = {'QuarterYr':'QuarterYr_Published_dt' })

# inx = [2461,3498,3701,4126,4165,4685,4990,5837,5930,7346,7632,7813,8379,8439,9167,9335,9400,9473,9813,9866,9867,9976,10195]
# data3 = data2.loc[inx]
# data4 = data4.drop(['QuarterNbr_y', 'QuarterNbr_x'], axis=1)
# for i in range(0,len(data4)):
#     if  data4['is_cals'].values[i]==0:
#         data4['Actual_QuarterYr'].values[i] =    data4['QuarterYr_Published_dt'].values[i]
# #    if data2['data_diff'].values[i] <120 and data2['data_diff'].values[i]>0 and data2['next_currt_diff'].values[i] <120 and data2['next_currt_diff'].values[i]>0 or data2['next_publish_date'].values[i] == '2099-01-01' :
#     elif data4['is_cals'].values[i]!=0 and data4['data_diff'].values[i]>0: 
#         data4['Actual_QuarterYr'].values[i] =    data4['QuarterYr_Published_dt'].values[i]
        
#     else:
#         data4['Actual_QuarterYr'].values[i] =    data4['QuarterYr'].values[i]
#bb         print(i)



In [32]:
data2.columns
#data4 = pd.merge(data4,quarter_index,how = 'left',left_on = 'Previous_QuarterNbr',right_on = 'QuarterNbr')

Index(['input company name', 'About', 'Transcript page heading',
       'Transcript heading', 'Date published', 'Preamble',
       'transcript after preamble', 'Transcript link', 'Scraped date',
       'date_earning_call', 'published_date_time', 'date_earning_call_str',
       'DateEarningQuarterYr', 'key', 'Count', 'max_published_date',
       'key_quarter', 'Count_quarter', 'next_publish_date', 'def_date',
       'filing_date_2', 'is_cals', 'data_diff_temp', 'data_diff', 'QuarterYr',
       'Next_QuarterYr', 'next_currt_diff_temp', 'next_currt_diff',
       'Actual_QuarterYr'],
      dtype='object')

In [ ]:

from generatefeatures import GenerateFeatures  
from utils import TextUtils
textutils = TextUtils(stopword_file='data/stopwords.csv', use_spacy=True)
generatefeatures = GenerateFeatures(textutils=textutils, lm_filepath='data/LoughranMcDonald_MasterDictionary_2016.csv')

text = 'hello how are you, this is dubious'
print(generatefeatures.generate_features(text))

In [111]:
df = data2[1:1000]
df = df.rename(columns = {'input company name':'Company'})
df= df.rename(columns = {'transcript after preamble':'QA'})
#from QA_seperation import get_analysts_from_preamble


In [114]:

#  To get the count of executivess
def count_executive(executives_analysts):
    no_executive = len(list(executives_analysts)[0])
    return(no_executive)

#for i in range(0,len(x)):
#    no_executive = len(list(x[i])[0])
#    print(i)

# To get count of analyts
def count_analyst(executives_analysts):
    no_analysts = len(list(executives_analysts)[1])
    return(no_analysts)


In [117]:
def get_analysts_from_preamble(preamble):
    if not preamble:
        return
    try:
        lines_raw = list(map(lambda x: x.strip(), preamble.split('^')))
        lines = list(zip(range(len(lines_raw)), lines_raw))
        B_index = list(map(lambda x: x[0], filter(lambda x: x[1].startswith('~'), lines)))
        B_index += [len(lines)]
        executive_index = list(filter(lambda x: x[1].lower().startswith('~executive'), lines))[0][0]
        executive_index_range = executive_index, B_index[B_index.index(executive_index)+1]
        executives_lines = lines_raw[executive_index_range[0]+1: executive_index_range[1]]
        executives_lines = list(filter(bool, executives_lines))
        executives = list(map(lambda x: re.split('[\,\-\–]', x)[0].strip(), executives_lines))
        
        #get executive name and ceo,cfo
        cxo = list()
        for l in executives_lines:
            try: 
                designation = re.split('[\,\-\–]', l)[1].strip()
                if len(re.split('[\,\-\–]', l)[1:])>1:
                    designation = "-".join(re.split('[\,\-\–]', l)[1:]).strip()
            except:
                designation = l.split('–')[1].strip()
            
            cxo.append(designation)
            
    #   get analyst
        analysts_index = list(filter(lambda x: x[1].lower().startswith('~analyst'), lines))[0][0]
        analysts_index_range = analysts_index, B_index[B_index.index(analysts_index)+1]
        analysts_lines = lines_raw[analysts_index_range[0]+1: analysts_index_range[1]]
        analysts_lines = list(filter(bool, analysts_lines))
        analysts = list(map(lambda x: re.split('[\,\-\–]', x)[0].strip(), analysts_lines))

    except Exception as e:
        #print(traceback.print_exc)
        return None 
    
    return(executives,analysts,cxo)


In [144]:
df['executives_analysts'] = df.Preamble.apply(get_analysts_from_preamble)
#df['no_executive'] = df['executives_analysts'].apply(count_executive)
#df['no_analysts'] = df['executives_analysts'].apply(count_analyst)
df['combined']=df['Preamble'].astype(str)+df['QA'].astype(str)
df['Size'] = [len(x) for x in df.loc[:,'combined']] 
df['Size_Preamble']= [len(x) for x in df.loc[:,'Preamble']]
df['Size_QA']= [len(x) for x in df.loc[:,'QA']]


In [145]:
df['Questions_Features'] = df['QA'].apply(generatefeatures.generate_features)
df_q_features = df['Questions_Features'].apply(pd.Series)
pd.concat([df, df_q_features], axis=1)

"Netflix (NASDAQ:NFLX) Q4 2010 Earnings Call January 26, 2011  6:00 PM ET^ ~Executives~ ^David Wells - Chief Financial Officer^Erin Kasenchak - ^Deborah Crawford - Vice President of Investor Relations^Reed Hastings - Founder, Chairman, Chief Executive Officer, President and Member of Stock Option Committee^ ~Operator~ ^Good day, everyone, and Welcome to the Netflix Fourth Quarter 2010 Earnings Q&A Session. At this time, for opening remarks and introductions, I will turn the call over to Deborah Crawford, Vice President of Investor Relations. Please go ahead.^ ~Erin Kasenchak~ ^Thank you, and good afternoon. Welcome to the Netflix Fourth Quarter 2010 Earnings Q&A Session. We announced our financial results for the fourth quarter at approximately 1:05 p.m. Pacific time today. The shareholder letter and the Q4 financial results and the webcast of this Q&A session are all available at the company's Investor Relations website at ir.netflix.com. ^As is our standard practice, this call will c

In [ ]:
df = data2

df = df.rename(columns = {'input company name':'Company'})
df= df.rename(columns = {'transcript after preamble':'QA'})
#from QA_seperation import get_analysts_from_preamble
df = df[1:1000]

In [ ]:

def add_features(AD,features,quarters,data_sub):
    for feature in features:
        for var in  quarters:
            AD = pd.merge(AD,data_sub[['Company','QuarterYr',feature]],how = 'left',
                   left_on = ['Company', var],right_on = ['Company','QuarterYr']) 
            AD = AD.drop(['QuarterYr'],axis = 1)
            AD = AD.rename(columns={feature: var+'_'+feature})
    return(AD)

def add_features_dependent(AD,features,quarters,data_sub):
    for feature in features:
        for var in  quarters:
            AD = pd.merge(AD,data_sub[['Company','Actual_QuarterYr',feature]],how = 'left',
                   left_on = ['Company', var],right_on = ['Company','Actual_QuarterYr']) 
            AD = AD.drop(['Actual_QuarterYr'],axis = 1)
            AD = AD.rename(columns={feature: var+'_'+feature})
    return(AD)


In [ ]:
st = time.time()
# Please run the cyclic-quarters.py code to get Base_AD
df_raw1 = list(set(df['Company']))
CurrentQuarter  = 'Q22018'
AD = CreateBaseAD(df_raw1,CurrentQuarter)
# Define quarters
quarters = ['Quarter1','Quarter2','Quarter3','Quarter4']  
quarters_dep = ['NextQuarter1', 'NextQuarter2', 'NextQuarter3', 'NextQuarter4']

#Dependent
col_req = 'is_cals'
data_sub  = df.loc[:,['Company','Actual_QuarterYr',col_req]]
data_sub= data_sub.rename(columns = {'is_cals':'CALS'})
dv = ['CALS']
AD1 =  add_features_dependent(AD,dv,quarters_dep,data_sub)    



In [ ]:
readability_features = ['Num_sent','Padding','FE_idx','DC_idx','DC_dif_words',
'Smog_Grade_idx','FK_Grade_idx','CL_Grade_idx','ARI_Grade_idx',
'LW_Grade_idx','GFox_Grade_idx','Best_Grade_idx']


col_req = 'combined'
data_sub  = df.loc[:,['Company','QuarterYr',col_req]]
sub_ad  = readability(data_sub,col_req,readability_features)
AD1 =  add_features(AD1,readability_features,quarters,sub_ad)   
del(sub_ad,col_req,data_sub) 



In [ ]:

#AD with Readability features using Preamble
col_req = 'Preamble'
readability_features_preamble = [col_req+'_'+i for i in readability_features]
data_sub  = df.loc[:,['Company','QuarterYr',col_req]]
sub_ad = readability(data_sub,col_req,readability_features_preamble)
AD1 =  add_features(AD1,readability_features_preamble,quarters,sub_ad)    
del(sub_ad,col_req,data_sub) 



In [ ]:
#AD with Readability features using Questions & Answers
col_req = 'QA'
readability_features_preamble = [col_req+'_'+i for i in readability_features]
data_sub  = df.loc[:,['Company','QuarterYr',col_req]]
sub_ad = readability(data_sub,col_req,readability_features_preamble)
AD1 =  add_features(AD1,readability_features_preamble,quarters,sub_ad)    


In [ ]:
time.time()-st
pd.DataFrame(AD1).to_csv("samp_AD2.csv")

In [26]:
import os

In [27]:
os.getcwd()

'/media/affine/New Volume/Chubb/Github/Chubb'

In [24]:
pd.DataFrame(data3).to_csv("AD_check.csv")

In [79]:
from qaextraction import create_df_QA,get_q_a,get_analysts_from_preamble